In [1]:
import os
import time
import numpy as np
import csv
from keras import models, layers
from tensorflow.keras.datasets import mnist
from keras.utils import to_categorical

In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [3]:
# Definir la semilla aleatoria para reproducibilidad
#np.random.seed(0)

# Obtener una selección aleatoria de índices
#indices = np.random.permutation(len(train_images))

# Seleccionar las imágenes y etiquetas correspondientes
#train_images = train_images[indices]
#train_labels = train_labels[indices]

In [4]:
print("num_train_images: ",len(train_images))
print("num_test_images: ",len(test_images))

num_train_images:  60000
num_test_images:  10000


In [5]:
# Normalizar y cambiar el rango de los datos a 0-1
train_images = train_images / 255.0
test_images = test_images / 255.0

In [6]:
train_labels = to_categorical(train_labels, num_classes=10)
test_labels = to_categorical(test_labels, num_classes=10)

In [7]:
# Directorio donde se guardarán los modelos
model_dir = 'modelos_entrenados_100_epoch'
os.makedirs(model_dir, exist_ok=True)

In [8]:
# Lista para guardar los registros de cada modelo
model_metrics = []

for i in range(5):
    
    X = 64  # Número de filtros en la primera capa convolucional
    Y = 32  # Número de filtros en la segunda capa convolucional
    Z = 64  # Número de nodos en la capa completamente conectada (FC)

    model = models.Sequential([
        # Primera capa convolucional
        layers.Conv2D(X, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),

        # Segunda capa convolucional
        layers.Conv2D(Y, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        # Aplanar el output para alimentar la capa densa
        layers.Flatten(),

        # Capa completamente conectada (FC)
        layers.Dense(Z, activation='relu'),

        # Capa de salida (Readout)
        layers.Dense(10, activation='softmax')
    ])

    # Compilar el modelo
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Inicio del tiempo de entrenamiento
    start_time = time.time()

    # Entrenamiento del modelo
    history = model.fit(train_images, train_labels, epochs=100, batch_size=128, validation_data=(test_images, test_labels))

    # Tiempo total del entrenamiento
    training_time = time.time() - start_time

    # Precisión final del modelo
    final_accuracy = history.history['val_accuracy'][-1]

    # Guardar el modelo
    model_path = os.path.join(model_dir, f'modelo_{i+1}.h5')
    model.save(model_path)
    print(f'Modelo {i+1} guardado en {model_path}')

    # Guardar métricas en la lista
    model_metrics.append((training_time, final_accuracy))

Epoch 1/100
469/469 [==============================] - 23s 47ms/step - loss: 0.2551 - accuracy: 0.9247 - val_loss: 0.0606 - val_accuracy: 0.9806
Epoch 2/100
469/469 [==============================] - 24s 50ms/step - loss: 0.0664 - accuracy: 0.9795 - val_loss: 0.0527 - val_accuracy: 0.9842
Epoch 3/100
469/469 [==============================] - 23s 50ms/step - loss: 0.0458 - accuracy: 0.9858 - val_loss: 0.0399 - val_accuracy: 0.9878
Epoch 4/100
469/469 [==============================] - 22s 47ms/step - loss: 0.0360 - accuracy: 0.9889 - val_loss: 0.0352 - val_accuracy: 0.9882
Epoch 5/100
469/469 [==============================] - 23s 48ms/step - loss: 0.0298 - accuracy: 0.9908 - val_loss: 0.0366 - val_accuracy: 0.9876
Epoch 6/100
469/469 [==============================] - 20s 43ms/step - loss: 0.0249 - accuracy: 0.9924 - val_loss: 0.0387 - val_accuracy: 0.9870
Epoch 7/100
469/469 [==============================] - 18s 38ms/step - loss: 0.0206 - accuracy: 0.9934 - val_loss: 0.0328 - val_ac

KeyboardInterrupt: 

In [ ]:
print("ID del Modelo\tTiempo de Entrenamiento (s)\tPrecisión de Validación")
for i, metrics in enumerate(model_metrics):
    print(f"{i+1}\t\t{metrics[0]:.2f}\t\t\t\t{metrics[1]:.4f}")

ID del Modelo	Tiempo de Entrenamiento (s)	Precisión de Validación
1		406.00				0.9910
2		420.52				0.9890
3		416.92				0.9877
4		428.14				0.9917
5		425.16				0.9897
6		430.07				0.9896
7		425.16				0.9888
8		427.59				0.9900
9		382.70				0.9884
10		374.49				0.9905


In [ ]:
# Nombre del archivo CSV donde se guardarán las métricas
metrics_file = os.path.join(model_dir, 'model_metrics.csv')


# Guardar la tabla de métricas en un archivo CSV
with open(metrics_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Escribir los encabezados del CSV
    writer.writerow(['ID del Modelo', 'Tiempo de Entrenamiento (s)', 'Precisión de Validación'])
    # Escribir las métricas de cada modelo
    for metrics in model_metrics:
        writer.writerow(metrics)